<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/2_OpenAI_SQLAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation


In [12]:
!pip install openai -q
!pip install supabase -q
!pip install pyodbc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 6.7 MB/s eta 0:00:00


## Get API Keys and setup

In [31]:
from openai import OpenAI

from supabase import create_client, Client

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

import pandas as pd
import pyodbc

import requests
import json
from google.colab import userdata

# setup the gemini pro
# gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key=userdata.get('Gemini_API_Key'))

# setup Supabase connection
supabase_url = userdata.get('SUPABASE_URL')
supabase_key = userdata.get('SUPABASE_KEY')
supabase_client = create_client(supabase_url, supabase_key)

# OpenAI key
openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = OpenAI(api_key=openai_api_key)


## Helper functions to run code


In [16]:
#
# Retrieve a file from Github Repository

def get_ChinookDB_ER_from_github(file_url):

  # Send a GET request to the file URL
  response = requests.get(file_url)

  # Check if the request was successful
  if response.status_code == 200:
      # Get the content of the file
      file_content = response.text
      return file_content
  else:
      return(f'Failed to retrieve the file: {response.status_code}')

## Test connection to
- Google Gemini
- Supabase
- [https://github.com/lerocha/chinook-database](https://github.com/lerocha/chinook-database)

In [14]:
## Testing connection to Supabase using Supabase client. A simple sanity check

response = supabase_client.table('Customer').select("*").execute()

## response object has data attribute that contains the result
## response.data
response_df = pd.DataFrame(response.data)
response_df.head(3)


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


## Get OpenAI assistant

In [56]:
## OpenAI assistant and attached file details are shown at the end of this notebook

# Get OpenAI assistant
db_assistant = openai_client.beta.assistants.retrieve("asst_v3MqBfnVWdywefUUKPTIIB0P")

# Start a new thread to chat with OpenAI
thread = openai_client.beta.threads.create()

# Let's ask the first question - How many employees are there?
thread_message = openai_client.beta.threads.messages.create(
    thread_id = thread.id,
    role="user",
    content="How many employees are there?",
    )
run = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=db_assistant.id)

In [58]:
# Let's ask another question - How many employees are there?
thread_message = openai_client.beta.threads.messages.create(
    thread_id = thread.id,
    role="user",
    content="Give me total sales by genre for all genres",
    )
run = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=db_assistant.id)

In [59]:
# Retrieve the chat history

history = openai_client.beta.threads.messages.list(thread_id=thread.id)

messages = history.data[::-1]
for i in messages:
    if(i.role.upper() == "USER"):
      print("\n","*"*60)
    print(i.role.upper() + ": "+ i.content[0].text.value)


 ************************************************************
USER: How many employees are there?
ASSISTANT: The ER diagram script indicates the structure of several tables, including `Album`, `Artist`, `Customer`, and likely more as the provided script snippet is limited. From this snippet, there isn't a direct reference to an `Employee` table. I will now search the entire script content for references to an `Employee` table to accurately respond to your query.
ASSISTANT: The ER diagram script contains information about an `Employee` table. To answer your query about the number of employees, I will provide the SQL query needed to count the number of employees in the `Employee` table.

```json
{
  "SQL code": "SELECT COUNT(*) FROM \"Employee\";",
  "Explanation": "This SQL query counts the total number of rows present in the 'Employee' table, which corresponds to the number of employees. The table name is enclosed in double quotes to ensure it is correctly interpreted by Postgres in c

In [60]:
print(history.model_dump_json(indent=4))


{
    "data": [
        {
            "id": "msg_pnUQFb9qla7BHDeFFd5qVhyb",
            "assistant_id": "asst_v3MqBfnVWdywefUUKPTIIB0P",
            "completed_at": null,
            "content": [
                {
                    "text": {
                        "annotations": [],
                        "value": "Based on the analysis of the ER diagram script, we have identified the relevant tables for calculating total sales by genre:\n\n- `Genre`: This table likely holds the different genres.\n- `Track`: This table holds information related to individual tracks, which should include a relationship to genres.\n- `Invoice`: This table likely contains the sales transactions.\n- `InvoiceLine`: This table likely contains details about each item (such as a track) sold in an invoice, including quantities and prices.\n\nGiven this structure, we can construct a SQL query to join these tables to calculate total sales by genre. The precise relationships between these tables (such as forei

In [ ]:
# Inspect db_assistant
# print(db_assistant)
# print(db_assistant.data)
# print(db_assistant.model_dump_json(indent=4))

In [32]:
# Display OpenAI Assistant info

print(db_assistant.model_dump_json(indent=4))

{
    "id": "asst_v3MqBfnVWdywefUUKPTIIB0P",
    "created_at": 1711830731,
    "description": null,
    "file_ids": [
        "file-nYMstH2zqNqOYe8N4YXqV7iI"
    ],
    "instructions": "You are a Data Architect for ChinookDB. The mermaid ER script is in the attached file. Load and analyze the ER  script from the file provided by me. I will ask you to write queries and you will use this ER diagram to deliver SQL queries. Database is postgres. use quotes for database object names in SQL",
    "metadata": {},
    "model": "gpt-4-turbo-preview",
    "name": "ChinookDB",
    "object": "assistant",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ]
}


In [33]:
chinookDB_ER = get_ChinookDB_ER_from_github('https://raw.githubusercontent.com/cooolbabu/GoogleGemini101/main/OpenAI/ChinookDB_ER.txt')
print(chinookDB_ER)

erDiagram
    Album {
        integer AlbumId PK
        varchar Title
        integer ArtistId FK
    }

    Artist {
        integer ArtistId PK
        varchar Name
    }

    Customer {
        integer CustomerId PK
        varchar FirstName
        varchar LastName
        varchar Company
        varchar Address
        varchar City
        varchar State
        varchar Country
        varchar PostalCode
        varchar Phone
        varchar Fax
        varchar Email
        integer SupportRepId FK
    }

    Employee {
        integer EmployeeId PK
        varchar LastName
        varchar FirstName
        varchar Title
        integer ReportsTo FK
        datetime BirthDate
        datetime HireDate
        varchar Address
        varchar City
        varchar State
        varchar Country
        varchar PostalCode
        varchar Phone
        varchar Fax
        varchar Email
    }

    Genre {
        integer GenreId PK
        varchar Name
    }

    Invoice {
        intege